<h1><center> Limpieza de Datos<h1>

## Librerias

In [16]:
import pandas as pd
from functools import reduce

## Cargar Datos

In [17]:
pm25      = pd.read_csv("Datos/MP 2.5.csv",           sep=";", decimal=",")
pm10      = pd.read_csv("Datos/MP10.csv",             sep=";", decimal=",")
so2       = pd.read_csv("Datos/Dioxido de azufre.csv",sep=";", decimal=",")
co        = pd.read_csv("Datos/monoxido de carbono.csv", sep=";", decimal=",")
no2       = pd.read_csv("Datos/NO2.csv",              sep=";", decimal=",")
o3        = pd.read_csv("Datos/O3.csv",               sep=";", decimal=",")
viento    = pd.read_csv("Datos/VIENTO.csv",           sep=";", decimal=",")
viento_ms = pd.read_csv("Datos/VIENTO MS.csv",        sep=";", decimal=",")

In [18]:
lista = [pm25, pm10, so2, co, no2, o3, viento, viento_ms]
for df in lista: # Para ver las dimensiones de cada df
    print(df.shape)

(3610, 6)
(3941, 6)
(3599, 6)
(3599, 6)
(4057, 6)
(4057, 6)
(24, 4)
(117167, 4)


In [19]:
so2.head()

,FECHA (YYMMDD),HORA (HHMM),Registros validados,Registros preliminares,Registros no validados,Unnamed: 5
0,160201,0,NaN,NaN,0.076310,NaN
1,160202,0,NaN,NaN,0.161685,NaN
2,160203,0,NaN,NaN,0.369947,NaN
3,160204,0,NaN,NaN,0.316450,NaN
4,160205,0,NaN,NaN,0.336140,NaN


In [20]:
co.head()

,FECHA (YYMMDD),HORA (HHMM),Registros validados,Registros preliminares,Registros no validados,Unnamed: 5
0,160201,0,NaN,NaN,0.076310,NaN
1,160202,0,NaN,NaN,0.161685,NaN
2,160203,0,NaN,NaN,0.369947,NaN
3,160204,0,NaN,NaN,0.316450,NaN
4,160205,0,NaN,NaN,0.336140,NaN


## Limpiar Datos

### Parámetros contaminantes

In [21]:
def preparar_contaminante(df_raw, nombre_valor):
    df = df_raw.copy()

    # --- FECHA ---
    # Si viene como número (160121) lo pasamos a string con 6 dígitos
    if pd.api.types.is_numeric_dtype(df["FECHA (YYMMDD)"]):
        df["FECHA (YYMMDD)"] = (
            df["FECHA (YYMMDD)"]
            .astype("Int64")   # entero "con NA"
            .astype(str)
            .str.zfill(6)
        )
    else:
        df["FECHA (YYMMDD)"] = df["FECHA (YYMMDD)"].astype(str).str.zfill(6)

    # --- HORA ---
    df["HORA (HHMM)"] = df["HORA (HHMM)"].astype(str).str.zfill(4)

    # --- fecha_hora ---
    df["fecha_hora"] = pd.to_datetime(
        df["FECHA (YYMMDD)"] + df["HORA (HHMM)"],
        format="%y%m%d%H%M",
        errors="coerce"
    )

    # --- valor: primero validados, luego preliminares, luego no validados ---
    posibles_cols = [
        col for col in [
            "Registros validados",
            "Registros preliminares",
            "Registros no validados"
        ]
        if col in df.columns
    ]

    # bfill por filas: toma el primer no-NA de esas columnas
    df[nombre_valor] = df[posibles_cols].bfill(axis=1).iloc[:, 0]

    # Nos quedamos solo con fecha_hora y la variable, ordenado y sin NA en el valor
    df = (
        df[["fecha_hora", nombre_valor]]
        .dropna(subset=["fecha_hora", nombre_valor])
        .sort_values("fecha_hora")
        .reset_index(drop=True)
    )

    return df


In [22]:
pm10_clean = preparar_contaminante(pm10, "pm10")
so2_clean  = preparar_contaminante(so2,  "so2")
co_clean   = preparar_contaminante(co,   "co")
no2_clean  = preparar_contaminante(no2,  "no2")
o3_clean   = preparar_contaminante(o3,   "o3")
pm25_clean = preparar_contaminante(pm25, "pm25")

In [ ]:
# Importante co y so2 son iguales
column_equal = co['Registros no validados'].equals(so2['Registros no validados'])

if column_equal:
    print("Las columnas 'Registros no validados' son iguales.")


Las columnas 'Registros no validados' son iguales.


### Parámetros meteorológicos

In [24]:
def preparar_meteo(df_raw, nombre_valor):
    df = df_raw.copy()

    # --- FECHA ---
    if pd.api.types.is_numeric_dtype(df["FECHA (YYMMDD)"]):
        df["FECHA (YYMMDD)"] = (
            df["FECHA (YYMMDD)"]
            .astype("Int64")
            .astype(str)
            .str.zfill(6)
        )
    else:
        df["FECHA (YYMMDD)"] = df["FECHA (YYMMDD)"].astype(str).str.zfill(6)

    # --- HORA ---
    df["HORA (HHMM)"] = df["HORA (HHMM)"].astype(str).str.zfill(4)

    # --- fecha_hora ---
    df["fecha_hora"] = pd.to_datetime(
        df["FECHA (YYMMDD)"] + df["HORA (HHMM)"],
        format="%y%m%d%H%M",
        errors="coerce"
    )

    # columnas de valor = todas las que vienen después de FECHA y HORA
    valor_cols = df.columns[2:]

    # usamos la primera de esas columnas como valor de la serie
    col_valor = valor_cols[0]

    df[nombre_valor] = pd.to_numeric(df[col_valor], errors="coerce")

    # nos quedamos solo con fecha_hora + valor, ordenado y sin NA
    df = (
        df[["fecha_hora", nombre_valor]]
        .dropna(subset=["fecha_hora", nombre_valor])
        .sort_values("fecha_hora")
        .reset_index(drop=True)
    )

    return df


In [25]:
viento_clean    = preparar_meteo(viento,    "viento")
viento_ms_clean = preparar_meteo(viento_ms, "viento_ms")

viento_ms_clean.head()

,fecha_hora,viento_ms
0,2012-07-30 16:00:00,1.492890
1,2012-07-30 17:00:00,1.447190
2,2012-07-30 18:00:00,1.158750
3,2012-07-30 19:00:00,0.653731
4,2012-07-30 20:00:00,1.503220


el paramentro **VIENTO** no nos sirve, (no hay suficientes datos) 

Es: **viento_clean**

## Guardar Datos

In [26]:
dfs = [
    pm25_clean,
    pm10_clean,
    so2_clean,
    co_clean,
    no2_clean,
    o3_clean,
    viento_ms_clean, 
]

datos = reduce(
    lambda left, right: pd.merge(left, right, on="fecha_hora", how="inner"),
    dfs
)

datos = datos.sort_values("fecha_hora").reset_index(drop=True)

print(datos.shape) # dimensiones del df final
print(datos.isna().sum())


(3462, 8)
fecha_hora    0
pm25          0
pm10          0
so2           0
co            0
no2           0
o3            0
viento_ms     0
dtype: int64


In [27]:
datos.head()

,fecha_hora,pm25,pm10,so2,co,no2,o3,viento_ms
0,2016-02-01,11.3639,49.4767,0.076310,0.076310,2.62036,6.77202,1.47277
1,2016-02-02,17.1773,50.3477,0.161685,0.161685,2.23645,9.22595,2.10613
2,2016-02-03,16.1939,41.5487,0.369947,0.369947,2.86062,10.70280,3.41691
3,2016-02-04,13.5039,66.9998,0.316450,0.316450,2.30634,10.21150,1.31098
4,2016-02-05,14.3462,62.6202,0.336140,0.336140,1.92460,10.94380,1.50804


In [28]:
datos.to_csv("Datos/datos_combinados.csv", index=False)